# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

## Изучим файл с данными и общую информацию

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("/file.csv")

display(data.head(10))
data.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод** В файле с данными о заемщиках, есть пропуски и возможно неправильный тип данных.

## Предобработка данных

### Обработка пропусков

In [3]:
# Изучим столбец "days_employed"
display(data.days_employed.describe())

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

In [4]:
# Приведем столбец "days_employed" в днях к нормальному виду (будем считать , что значение больше 22000 существовать не может,
# соотвественно это часы, переведем эти занчения в дни)
data.loc[(data['days_employed'] >22000), 'days_employed'] = data.loc[(data['days_employed'] >22000), 'days_employed'] / 24

In [5]:
# Округлим зарплату до сотых
data['total_income'] = round(data['total_income'].abs(), 2)

In [6]:
# В колонках 'days_employed' и 'total_income' есть пропуски, поправим это:
data.loc[data['days_employed'].isna(),'days_employed'] = data['days_employed'].median()
data['days_employed'] = data['days_employed'].abs().astype('int') # округлим и уберем отрицательные значения
print('Пропусков в колонке days_employed: ', data['days_employed'].isna().sum())

Пропусков в колонке days_employed:  0


In [7]:
# Cформируем сводную табличку с группировкой по типу занятости и измерением медианного дохода
group_income = data.groupby('income_type')['total_income'].median()
# заполним пропуски на медианный доход, основываясь на типе занятости
for i in group_income.index: 
    data.loc[(data['income_type'] == i) & (data['total_income'].isnull()), 'total_income'] = group_income[i]
    
print('Пропусков в колонке total_income: ',data['total_income'].isna().sum())

Пропусков в колонке total_income:  0


In [8]:
data.info() # Снова проверим датафрейм

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  int64  
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


**Вывод** 

Прежде чем заполнять пропуски по стажу ('days_employed'), приведем данные в порядок. 
- Положительные значения слишком велики, скорее всего они в часах, для нашей выборки я считаю что больше 22000 дней стаж быть не может. 
- Отрицательные выглядят одеватно и эти данные просто возьмем по модклю. 

Для заполнения пропусков в колонке доход ('income_type') сгруппируем данные по типу занятости и относительно этих групп заполним медианными занчениями пропуски.


Убираем пропуски и теперь видно, что в колонках 'days_employed' и 'total_income' пропуски заменены на средние значения

### Замена типа данных

In [9]:
# Видно, что данные не однородны: разный регистр,возможно единица измерения,отрицательные данные, неверные данные. 
# Исправим это:
# С помощью цикла приведем строки к нижнему регистру
for col in data:
    try:
        data[col] = data[col].str.lower()
    except AttributeError:
        pass
    
data['gender'] = data['gender'].str.upper() # Столбец гендер сделаем с верхним регистром

# Поправим значения в колорнке 'children'        
data.loc[(data['children'] == 20), 'children'] = 2 # предположим опечатка
data.loc[(data['children'] == -1), 'children'] = 0 # предположим системная ошиюка

# Поправим занчения в колонке 'gender'
data.loc[(data['gender'] == 'XNA'), 'gender'] = 'F' # поменяем некорректное занчение на значение 

# Поправим занчения  "0" в колонке 'dob_years' на средний возраст
data.loc[(data['dob_years'] == 0), 'dob_years'] = data['dob_years'].mean().astype('int')

# Заменим вещественный тип данных в столбце total_income на целочисленный
data['total_income'] = data['total_income'].astype('int')

display(data)
print(data.info())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21521,0,14330,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB
None


**Вывод** Теперь данные более однородны и приведены в порядок

### Работа с дубликатами

In [10]:
# Проверим таблицу на дубликаты и удалим их.
print('Количество дубликатов до: ', data.duplicated().sum())

data = data.drop_duplicates() #удаляем дубликаты

print('Количество дубликатов после: ', data.duplicated().reset_index(drop = True).sum())


Количество дубликатов до:  71
Количество дубликатов после:  0


**Вывод** Дубликаты удалены.

### Лемматизация

In [11]:
# В столбце "purpose" одни и те же данные могут называться по разному или похоже, проверим.
# Применим Лемматизацию к столбцу "purpose"
from pymystem3 import Mystem
m = Mystem() 

lemmas = m.lemmatize(' '.join(data['purpose'].unique()))
from collections import Counter
Counter(lemmas)

Counter({'покупка': 10,
         ' ': 96,
         'жилье': 7,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 2,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})

**Вывод**  Видно, что данные указывались в свободном стиле и нуждаются в категоризации т.к. одно и тоже может называться по разному.

### Категоризация данных

In [12]:
# Напишем функцию для разбивки по лемам на основные категории
def category(value):
    lemmas = m.lemmatize(value)
    if 'недвижимость' in lemmas:
        return 'недвижимость'
    if 'жилье' in lemmas:
        return 'недвижимость'    
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    if 'образование' in lemmas:
        return 'образование'
    if 'свадьба' in lemmas:
        return 'свадьба'
    return 'прочее'

# Добавим новый столбец
data['purpose_category'] = data['purpose'].apply(category)

display(data)
print(data['purpose_category'].value_counts())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,недвижимость
21521,0,14330,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,автомобиль
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль


недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_category, dtype: int64


**Вывод** Теперь все выглядит аккуратно и разбито на категории. Видно , что Заемщики чаще всего берут кредиты для сделок с недвижимостью. А в категорию "Прочее" не чего не вошло.

## Ответим на вопросы

**1. Есть ли зависимость между наличием детей и возвратом кредита в срок?**

In [13]:
# напишем функцию для разделения на категории
def categ(value):
    if value == 0:
        return 'Заёмщики без детей'
    else:
        return 'Заёмщики с детьми'

# Добавим новый столбец
data['children_category'] = data['children'].apply(categ)
# сгруппируем таблицу     
report = data.pivot_table(index = 'children_category', columns = 'debt', values = 'total_income', aggfunc = 'count')
report.columns = ['no debt', 'debt']
# добавим доли
report['%'] = round(report['debt']*100/(report['debt'] +  report['no debt']), 2)
display(report)

,no debt,debt,%
children_category,,,
Заёмщики без детей,13074,1064,7.53
Заёмщики с детьми,6639,677,9.25


**Вывод** Судя по сводной таблице процент не возвратов в срок среди заемщиков с детьми выше.

**2. Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [14]:
# сгруппируем таблицу по семейному статусу
report_family = data.pivot_table(index = 'family_status', columns = 'debt', values = 'total_income', aggfunc = 'count')
report_family.columns = ['no debt', 'debt']
# добавим доли
report_family['%'] = round(report_family['debt']*100/(report_family['debt'] +  report_family['no debt']), 2)
display(report_family)

,no debt,debt,%
family_status,,,
в разводе,1110,85,7.11
вдовец / вдова,896,63,6.57
гражданский брак,3763,388,9.35
женат / замужем,11408,931,7.55
не женат / не замужем,2536,274,9.75


**Вывод** Из сводной таблицы следует, что заемщики со статусами "Гражданский брак" и "Не женат/ не замужем"  чаще остальных групп не возвращают кредит в срок. Статус официальности отношений действительно положительно сказывается на возврате кредита. Люди, которые не состоят в официальных отношениях и никогда не состояли хуже возвращают кредиты, в отличии от семей, которые имеют официальный статус, пусть даже когда-то в прошлом.

**3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [15]:
# проведем категоризацию, а именно разбьем на группы, опираясь на квартили распределения
data['total_income_gr'] = pd.qcut(data['total_income'], 4, labels=["низкий доход", "средний доход", " доход выше среднего", "высокий доход"])
# сгруппируме таблицу
report_income = data.pivot_table(index = 'total_income_gr', columns = 'debt', values = 'total_income', aggfunc = 'count')
report_income.columns = ['no debt', 'debt']
# добавим доли
report_income['%'] = round(report_income['debt']*100/(report_income['debt'] +  report_income['no debt']), 2)
display(report_income)

,no debt,debt,%
total_income_gr,,,
низкий доход,4937,427,7.96
средний доход,4996,483,8.82
доход выше среднего,4799,448,8.54
высокий доход,4981,383,7.14


**Вывод** Судя по таблице видно, что Заемщики со Средним доходом чаще остальных групп не возвращают кредит в срок. Скорее всего люди с низким доходом просто не берут кредит на то, чего не смогут вернуть, проходят куда более строгий контроль; люди с высоким достатком просто обладают достаточными средствами на погашение; люди со средними доходами вероятно слишком оптимистично подходят к своим финансовыми возможностям

**4. Как разные цели кредита влияют на его возврат в срок?**

In [16]:
# сгруппируме таблицу
report_purpose = data.pivot_table(index = 'purpose_category', columns = 'debt', values = 'total_income', aggfunc = 'count')
report_purpose.columns = ['no debt', 'debt']
# добавим доли
report_purpose['%'] = round(report_purpose['debt']*100/(report_purpose['debt'] +  report_purpose['no debt']), 2)
display(report_purpose)

,no debt,debt,%
purpose_category,,,
автомобиль,3903,403,9.36
недвижимость,10029,782,7.23
образование,3643,370,9.22
свадьба,2138,186,8.00


**Вывод** Изучив сводную таблицу напрашивается вывод, что недвижимость, как правило находится в залоге у банка, к тому же люди осознанно подходят к взятию кредита на данную цель; образование вероятно не сразу окупается, тк студент должен набраться опыта; автомобиль влечет за собой множество дополнительных расходов, о которых люди часто забывают, когда берут на него кредит

## Общий вывод 
Влияние количества детей и семейного положения на заемщика безусловно присуствует, что доказывает данная выборка. 
Приведя данные в порядок и используя не хитрые группировки данных с помощью сводных таблиц удалось убедтся, что заемщик не имеющий детей и находящий в семейном статусе "Вдовец/вдова", "женат / замужем" и "В разводе" более надежные остальных.
И связано это скорее всего с наименьшей финансовой нагрузкой на заемщика. Доход заемщиков также влияет на возращения кредита в срок. Более надежные являтся заемщики с низким и высоким доходом, это следует скорее всего из того, что люди с низким доходом проходят более тчательный контроль; люди с высоким достатком обладают достаточными средствами на погашение